In [1]:
import requests
import geopandas as gpd
import os

# Path to the shapefile
shapefile_path = "../data/city_boundaries/Chicago"

# Read the shapefile
shapefile = gpd.read_file(shapefile_path)

# Extract the bounding box coordinates
bbox = shapefile.total_bounds

# Construct the API request URL
api_url = "https://overpass-api.de/api/interpreter"  # OSM Overpass API endpoint
query = f"[out:json];(node['image'='satellite']({','.join(str(coord) for coord in bbox)});way['image'='satellite']({','.join(str(coord) for coord in bbox)});relation['image'='satellite']({','.join(str(coord) for coord in bbox)}););out body;>;out skel qt;"

# Send the API request
response = requests.get(api_url, params={"data": query})

# Check if the request was successful
if response.status_code == 200:
    # Process the response data
    response_data = response.json()
    
    # Check if there are elements in the response
    if "elements" in response_data and len(response_data["elements"]) > 0:
        # Extract and save the satellite imagery from the response
        for element in response_data["elements"]:
            if element["type"] == "node" or element["type"] == "way" or element["type"] == "relation":
                tags = element.get("tags", {})
                image_url = tags.get("image")
                if image_url:
                    # Download the image
                    image_response = requests.get(image_url)
                    if image_response.status_code == 200:
                        # Save the image with a unique filename
                        filename = os.path.basename(image_url)
                        save_path = os.path.join("output_folder", filename)
                        with open(save_path, "wb") as file:
                            file.write(image_response.content)
                        print(f"Saved image: {save_path}")
                    else:
                        print(f"Failed to download image: {image_url}")
    else:
        print("No satellite imagery elements found in the response.")
else:
    # Handle the request failure
    print("API request failed with status code:", response.status_code)

No satellite imagery elements found in the response.
